In [1]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.vgg16 import preprocess_input, VGG16
from tensorflow.keras import datasets
import tensorflow.keras as K

def preprocess_data(X, y):
        X = X.astype('float32')
        # here i used the preprocess_input function from VGG16 to normalize the input data
        preprocessed_X = preprocess_input(X)
        # Transforming labels to correct format
        preprocessed_y = to_categorical(y, 10)
        return (preprocessed_X, preprocessed_y)

if __name__ == "__main__":
    #load data
    (X_train, y_train), (X_test, y_test) = datasets.cifar10.load_data()

    # preprocessing the data (train & test data)
    (X_train_p , y_train_p) = preprocess_data(X_train, y_train)
    (X_test_p , y_test_p) = preprocess_data(X_test, y_test)

    # Loading VGG16 model
    vgg16_model = VGG16(include_top=False,
                       weights='imagenet',
                       pooling='avg')
    
    model= K.Sequential()
    model.add(K.layers.UpSampling2D())
    model.add(vgg16_model)
    model.add(K.layers.Flatten())
    model.add(K.layers.Dense(512, activation=('relu'))) 
    model.add(K.layers.Dropout(0.2))
    model.add(K.layers.Dense(256, activation=('relu')))
    model.add(K.layers.Dropout(0.2))
    model.add(K.layers.Dense(10, activation=('softmax')))
    callback = []
    def decay(epoch):
        """ This method create the alpha"""
        return 0.001 / (1 + 1 * 30)
    callback += [K.callbacks.LearningRateScheduler(decay, verbose=1)]
    callback += [K.callbacks.ModelCheckpoint('cifar10.h5',
                                             save_best_only=True,
                                             mode='min'
                                             )]
    model.compile(optimizer='adam', loss='categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(X_train_p , y_train_p,
              batch_size=128,
              validation_data=(X_test_p , y_test_p),
              epochs=20, shuffle=True,
              callbacks=callback,
              verbose=1)

In [2]:
test_loss, test_acc = model.evaluate(X_test_p , y_test_p, batch_size= 32, verbose=2)
print("test_loss = ", test_loss)
print("test_acc = ", test_acc)

In [5]:
import matplotlib.pyplot as plt
def plot_accuracy_functions(history):    
    plt.plot(history.history['accuracy'], label='accuracy')
    plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.ylim([0.5, 1])
    plt.legend(loc='lower right')
    
def plot_loss_functions(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

In [6]:
plot_accuracy_functions(history)

In [8]:
plot_loss_functions(history)

In [9]:
from tensorflow.keras import models
models.save_model(model,'VGG16_model.hdf5')

In [10]:
vgg16model = models.load_model('./VGG16_model.hdf5')

In [11]:
test_loss, test_acc = vgg16model.evaluate(X_test_p , y_test_p, batch_size= 32, verbose=2)
print("test_loss = ", test_loss)
print("test_acc = ", test_acc)